In [2]:
#importing and pre-processing data
import pandas as pd
import numpy as np
import re

In [3]:
# Read the data into a Pandas DataFrame
loan_info_df = pd.read_json('Resources/loan_approval_dataset.json')
loan_info_df.set_index('Id', inplace=True)
loan_info_df.drop_duplicates()
loan_info_df.head()


,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag
Id,,,,,,,,,,,,
1,1303834,23,3,single,rented,no,Mechanical_engineer,Rewa,Madhya_Pradesh,3,13,0
2,7574516,40,10,single,rented,no,Software_Developer,Parbhani,Maharashtra,9,13,0
3,3991815,66,4,married,rented,no,Technical_writer,Alappuzha,Kerala,4,10,0
4,6256451,41,2,single,rented,yes,Software_Developer,Bhubaneswar,Odisha,2,12,1
5,5768871,47,11,single,rented,no,Civil_servant,Tiruchirappalli[10],Tamil_Nadu,3,14,1


In [4]:
#Displaying the last few rows of the dataset
loan_info_df.tail()

,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag
Id,,,,,,,,,,,,
251996,8154883,43,13,single,rented,no,Surgeon,Kolkata,West_Bengal,6,11,0
251997,2843572,26,10,single,rented,no,Army_officer,Rewa,Madhya_Pradesh,6,11,0
251998,4522448,46,7,single,rented,no,Design_Engineer,Kalyan-Dombivli,Maharashtra,7,12,0
251999,6507128,45,0,single,rented,no,Graphic_Designer,Pondicherry,Puducherry,0,10,0
252000,9070230,70,17,single,rented,no,Statistician,Avadi,Tamil_Nadu,7,11,0


In [5]:
# Getting a concise summary of the dataset
loan_info_df.describe()
loan_info_df.info()
loan_info_df.columns

<class 'pandas.core.frame.DataFrame'>
Int64Index: 252000 entries, 1 to 252000
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Income             252000 non-null  int64 
 1   Age                252000 non-null  int64 
 2   Experience         252000 non-null  int64 
 3   Married/Single     252000 non-null  object
 4   House_Ownership    252000 non-null  object
 5   Car_Ownership      252000 non-null  object
 6   Profession         252000 non-null  object
 7   CITY               252000 non-null  object
 8   STATE              252000 non-null  object
 9   CURRENT_JOB_YRS    252000 non-null  int64 
 10  CURRENT_HOUSE_YRS  252000 non-null  int64 
 11  Risk_Flag          252000 non-null  int64 
dtypes: int64(6), object(6)
memory usage: 25.0+ MB


Index(['Income', 'Age', 'Experience', 'Married/Single', 'House_Ownership',
       'Car_Ownership', 'Profession', 'CITY', 'STATE', 'CURRENT_JOB_YRS',
       'CURRENT_HOUSE_YRS', 'Risk_Flag'],
      dtype='object')

In [6]:
# Checking for any missing values in the dataset
loan_info_df.isna().sum()

Income               0
Age                  0
Experience           0
Married/Single       0
House_Ownership      0
Car_Ownership        0
Profession           0
CITY                 0
STATE                0
CURRENT_JOB_YRS      0
CURRENT_HOUSE_YRS    0
Risk_Flag            0
dtype: int64

In [7]:
# Checking for value count in each state
loan_info_df['STATE'].value_counts()

Uttar_Pradesh        28400
Maharashtra          25562
Andhra_Pradesh       25297
West_Bengal          23483
Bihar                19780
Tamil_Nadu           16537
Madhya_Pradesh       14122
Karnataka            11855
Gujarat              11408
Rajasthan             9174
Jharkhand             8965
Haryana               7890
Telangana             7524
Assam                 7062
Kerala                5805
Delhi                 5490
Punjab                4720
Odisha                4658
Chhattisgarh          3834
Uttarakhand           1874
Jammu_and_Kashmir     1780
Puducherry            1433
Mizoram                849
Manipur                849
Himachal_Pradesh       833
Tripura                809
Uttar_Pradesh[5]       743
Chandigarh             656
Sikkim                 608
Name: STATE, dtype: int64

In [8]:
# Getting the value counts for each profession
profession = loan_info_df['Profession'].value_counts()
profession

Physician                     5957
Statistician                  5806
Web_designer                  5397
Psychologist                  5390
Computer_hardware_engineer    5372
Drafter                       5359
Magistrate                    5357
Fashion_Designer              5304
Air_traffic_controller        5281
Comedian                      5259
Industrial_Engineer           5250
Mechanical_engineer           5217
Chemical_engineer             5205
Technical_writer              5195
Hotel_Manager                 5178
Financial_Analyst             5167
Graphic_Designer              5166
Flight_attendant              5128
Biomedical_Engineer           5127
Secretary                     5061
Software_Developer            5053
Petroleum_Engineer            5041
Police_officer                5035
Computer_operator             4990
Politician                    4944
Microbiologist                4881
Technician                    4864
Artist                        4861
Lawyer              

In [9]:
# Getting the value counts for marital status
marital_status = loan_info_df['Married/Single'].value_counts()
marital_status

single     226272
married     25728
Name: Married/Single, dtype: int64

In [10]:
# Getting the value counts for risky loans
risk_status = loan_info_df['Risk_Flag'].value_counts()
risk_status


0    221004
1     30996
Name: Risk_Flag, dtype: int64

In [11]:
# Getting the value counts for home status
home_status = loan_info_df['House_Ownership'].value_counts()
home_status

rented          231898
owned            12918
norent_noown      7184
Name: House_Ownership, dtype: int64

In [12]:
# Define a function to clean and capitalize profession
def clean_and_capitalize_profession(profession):
    if isinstance(profession, str):
        profession = re.sub(r'\[.*?\]', '', profession)  # Remove trailing non-letter characters
        profession = profession.replace('_', ' ')  # Replace underscores with spaces
        profession = profession.strip().title() # Strip leading and trailing spaces and capitalize properly
        words = profession.split()
        if len(words) > 1:
            words[1] = words[1].capitalize()  # Capitalize the second word
        profession = ' '.join(words)
    return profession

# Apply the function to the 'city' column
loan_info_df['Profession'] = loan_info_df['Profession'].apply(clean_and_capitalize_profession)

# Check the cleaned DataFrame
loan_info_df.head()

,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag
Id,,,,,,,,,,,,
1,1303834,23,3,single,rented,no,Mechanical Engineer,Rewa,Madhya_Pradesh,3,13,0
2,7574516,40,10,single,rented,no,Software Developer,Parbhani,Maharashtra,9,13,0
3,3991815,66,4,married,rented,no,Technical Writer,Alappuzha,Kerala,4,10,0
4,6256451,41,2,single,rented,yes,Software Developer,Bhubaneswar,Odisha,2,12,1
5,5768871,47,11,single,rented,no,Civil Servant,Tiruchirappalli[10],Tamil_Nadu,3,14,1


In [13]:
# Define a function to clean and capitalize city names
def clean_city_name(city):
    if isinstance(city, str):
        city = re.sub(r'\[.*?\]', '', city)  # Remove trailing non-letter characters
        city = city.replace('_', ' ')  # Replace underscores with spaces
        city = city.strip().title() # Strip leading and trailing spaces and capitalize properly
        city = city.capitalize()  # Capitalize the first letter
    return city

# Apply the function to the 'city' column
loan_info_df['CITY'] = loan_info_df['CITY'].apply(clean_city_name)

# Check the cleaned DataFrame
loan_info_df.head()

,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag
Id,,,,,,,,,,,,
1,1303834,23,3,single,rented,no,Mechanical Engineer,Rewa,Madhya_Pradesh,3,13,0
2,7574516,40,10,single,rented,no,Software Developer,Parbhani,Maharashtra,9,13,0
3,3991815,66,4,married,rented,no,Technical Writer,Alappuzha,Kerala,4,10,0
4,6256451,41,2,single,rented,yes,Software Developer,Bhubaneswar,Odisha,2,12,1
5,5768871,47,11,single,rented,no,Civil Servant,Tiruchirappalli,Tamil_Nadu,3,14,1


In [14]:
# Define a function to clean and capitalize state names
def clean_state_name(state):
    if isinstance(state, str):
        state = re.sub(r'\[.*?\]', '', state)  # Remove trailing non-letter characters
        state = state.replace('_', ' ')  # Replace underscores with spaces
        state = state.strip().title() # Strip leading and trailing spaces and capitalize properly
        words = state.split()
        if len(words) > 1:
            words[1] = words[1].capitalize()  # Capitalize the second word
        state = ' '.join(words)
    return state

# Apply the function to the 'state' column
loan_info_df['STATE'] = loan_info_df['STATE'].apply(clean_state_name)

# Check the cleaned DataFrame
loan_info_df.head()

,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag
Id,,,,,,,,,,,,
1,1303834,23,3,single,rented,no,Mechanical Engineer,Rewa,Madhya Pradesh,3,13,0
2,7574516,40,10,single,rented,no,Software Developer,Parbhani,Maharashtra,9,13,0
3,3991815,66,4,married,rented,no,Technical Writer,Alappuzha,Kerala,4,10,0
4,6256451,41,2,single,rented,yes,Software Developer,Bhubaneswar,Odisha,2,12,1
5,5768871,47,11,single,rented,no,Civil Servant,Tiruchirappalli,Tamil Nadu,3,14,1


In [15]:
# Rename columns
loan_info_df.rename(columns={
    'Income': 'INCOME',
    'Age': 'AGE',
    'Experience': 'EXPERIENCE',
    'Married/Single': 'MARITAL_STATUS',
    'House_Ownership': 'HOUSE_OWNERSHIP',
    'Car_Ownership': 'CAR_OWNERSHIP',
    'Profession': 'PROFESSION',
    'Risk_Flag': 'RISK_FLAG'
}, inplace=True)

loan_info_df.head()

,INCOME,AGE,EXPERIENCE,MARITAL_STATUS,HOUSE_OWNERSHIP,CAR_OWNERSHIP,PROFESSION,CITY,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,RISK_FLAG
Id,,,,,,,,,,,,
1,1303834,23,3,single,rented,no,Mechanical Engineer,Rewa,Madhya Pradesh,3,13,0
2,7574516,40,10,single,rented,no,Software Developer,Parbhani,Maharashtra,9,13,0
3,3991815,66,4,married,rented,no,Technical Writer,Alappuzha,Kerala,4,10,0
4,6256451,41,2,single,rented,yes,Software Developer,Bhubaneswar,Odisha,2,12,1
5,5768871,47,11,single,rented,no,Civil Servant,Tiruchirappalli,Tamil Nadu,3,14,1


In [17]:
# Add an ID column
loan_info_df['ID'] = range(1, len(loan_info_df) + 1)

# Reorder the columns to make 'ID' the first column
cols = ['ID'] + [col for col in loan_info_df.columns if col != 'ID']
loan_info_df = loan_info_df[cols]

loan_info_df.head()

,ID,INCOME,AGE,EXPERIENCE,MARITAL_STATUS,HOUSE_OWNERSHIP,CAR_OWNERSHIP,PROFESSION,CITY,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,RISK_FLAG
Id,,,,,,,,,,,,,
1,1,1303834,23,3,single,rented,no,Mechanical Engineer,Rewa,Madhya Pradesh,3,13,0
2,2,7574516,40,10,single,rented,no,Software Developer,Parbhani,Maharashtra,9,13,0
3,3,3991815,66,4,married,rented,no,Technical Writer,Alappuzha,Kerala,4,10,0
4,4,6256451,41,2,single,rented,yes,Software Developer,Bhubaneswar,Odisha,2,12,1
5,5,5768871,47,11,single,rented,no,Civil Servant,Tiruchirappalli,Tamil Nadu,3,14,1


In [18]:
# Get a summary statistics for all columns in the dataset
loan_info_df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
ID,252000.0,NaN,NaN,NaN,126000.5,72746.278255,1.0,63000.75,126000.5,189000.25,252000.0
INCOME,252000.0,NaN,NaN,NaN,4997116.665325,2878311.013611,10310.0,2503015.0,5000694.5,7477502.0,9999938.0
AGE,252000.0,NaN,NaN,NaN,49.954071,17.063855,21.0,35.0,50.0,65.0,79.0
EXPERIENCE,252000.0,NaN,NaN,NaN,10.084437,6.00259,0.0,5.0,10.0,15.0,20.0
MARITAL_STATUS,252000,2,single,226272,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HOUSE_OWNERSHIP,252000,3,rented,231898,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CAR_OWNERSHIP,252000,2,no,176000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PROFESSION,252000,51,Physician,5957,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CITY,252000,316,Aurangabad,1543,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STATE,252000,28,Uttar Pradesh,29143,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# Get a summary statistics for numeric columns in the dataset
loan_info_df.describe().T

,count,mean,std,min,25%,50%,75%,max
ID,252000.0,1.260005e+05,7.274628e+04,1.0,63000.75,126000.5,189000.25,252000.0
INCOME,252000.0,4.997117e+06,2.878311e+06,10310.0,2503015.00,5000694.5,7477502.00,9999938.0
AGE,252000.0,4.995407e+01,1.706385e+01,21.0,35.00,50.0,65.00,79.0
EXPERIENCE,252000.0,1.008444e+01,6.002590e+00,0.0,5.00,10.0,15.00,20.0
CURRENT_JOB_YRS,252000.0,6.333877e+00,3.647053e+00,0.0,3.00,6.0,9.00,14.0
CURRENT_HOUSE_YRS,252000.0,1.199779e+01,1.399037e+00,10.0,11.00,12.0,13.00,14.0
RISK_FLAG,252000.0,1.230000e-01,3.284379e-01,0.0,0.00,0.0,0.00,1.0


In [20]:
# Export DataFrame as CSV files.
loan_info_df.to_csv("Resources/loan_dataset.csv", index=False)

In [21]:
# Drop columns
loan_train_df = loan_info_df.drop(["CITY", "CURRENT_HOUSE_YRS"], axis=1)

In [22]:
# Export DataFrame as CSV files for train purpose
loan_train_df.to_csv("Resources/loan_training_dataset.csv", index=False)